In [75]:
clear

In [76]:
import os

In [77]:
os.chdir("/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding")
os.getcwd()

'/Users/chris/Desktop/Dropbox/MasterThesisAFM/Coding'

In [78]:
import csv

In [79]:
#CREATE DICTIONARY FOR NODES
def reader(filename):
    for (lineno, line) in enumerate(open(filename)):
        if lineno > 0: # skip header
            yield line

filename = "input_nodes.csv"
(l_nodes, nodenames, beg, end, change) = zip(*( row for row in csv.reader(reader(filename))))

d_nodes = dict(zip(l_nodes, change))

#for k, v in dict_nodes.items():
    #print(k," ",v)



In [80]:
#{k:int(v) for k, v in dict_nodes.items()} #dictionary comprehension if change needed

In [81]:
import networkx as nx 
DG=nx.DiGraph()

In [82]:
#ADD SOURCE/SINK EDGES, by definition all nodes are included in edgelist
for k, v in d_nodes.items():
    
    if k.endswith('Cr') or int(v)<0:
        DG.add_edge('S',k)
        nx.set_edge_attributes(DG, 'capacity', {('S',k): abs(int(v))})
        print("from 'S' to", k, "with capacity =",abs(int(v)))

    if k.endswith('Dr') and int(v)>=0:
        DG.add_edge(k,"T")
        nx.set_edge_attributes(DG, 'capacity', {(k, 'T'): int(v)})
        print("from", k,"to 'T'","with capacity =",int(v))

from 7Dr to 'T' with capacity = 3
from 2Dr to 'T' with capacity = 1
from 'S' to 5Cr with capacity = 3
from 'S' to 3Dr with capacity = 1
from 4Dr to 'T' with capacity = 1
from 8Dr to 'T' with capacity = 3
from 1Dr to 'T' with capacity = 3
from 'S' to 6Cr with capacity = 7


In [83]:
nx.edges(DG) #check edges that for source/sink nodes

[('7Dr', 'T'),
 ('2Dr', 'T'),
 ('S', '3Dr'),
 ('S', '5Cr'),
 ('S', '6Cr'),
 ('4Dr', 'T'),
 ('8Dr', 'T'),
 ('1Dr', 'T')]

In [84]:
#ADD TRANSACTION EDGES
def reader(filename2):
    for (lineno, line) in enumerate(open(filename2)):
        if lineno > 0: # skip header
            yield line

filename2 = "input_edges.csv"
(from_, to_) = zip(*( row for row in csv.reader(reader(filename2))))

l_edges = list(zip(from_, to_))

for f,t in l_edges:
    DG.add_edge(f,t)

In [85]:
nx.edges(DG) #check edges again

[('7Dr', 'T'),
 ('2Dr', '1Dr'),
 ('2Dr', 'T'),
 ('2Dr', '8Dr'),
 ('S', '3Dr'),
 ('S', '5Cr'),
 ('S', '6Cr'),
 ('4Dr', '3Dr'),
 ('4Dr', '8Dr'),
 ('4Dr', 'T'),
 ('3Dr', '7Dr'),
 ('5Cr', '3Dr'),
 ('5Cr', '8Dr'),
 ('8Dr', 'T'),
 ('6Cr', '2Dr'),
 ('1Dr', '4Dr'),
 ('1Dr', 'T'),
 ('1Dr', '5Cr')]

In [86]:
print('no. of nodes:',DG.number_of_nodes())
print('no. of edges:',DG.number_of_edges())
print('maximum flow:',nx.maximum_flow_value(DG, s='S', t='T'))
print('minimum cut: ',nx.minimum_cut_value(DG, s="S", t="T"))

no. of nodes: 10
no. of edges: 18
maximum flow: 11
minimum cut:  11


In [87]:
#run max-flow min cut algorithm
data = nx.maximum_flow(DG, s="S", t="T")
nx.maximum_flow(DG, s="S", t="T")

(11,
 {'1Dr': {'4Dr': 3, '5Cr': 0, 'T': 3},
  '2Dr': {'1Dr': 6, '8Dr': 0, 'T': 1},
  '3Dr': {'7Dr': 3},
  '4Dr': {'3Dr': 2, '8Dr': 0, 'T': 1},
  '5Cr': {'3Dr': 0, '8Dr': 3},
  '6Cr': {'2Dr': 7},
  '7Dr': {'T': 3},
  '8Dr': {'T': 3},
  'S': {'3Dr': 1, '5Cr': 3, '6Cr': 7},
  'T': {}})

In [88]:
y1 = []

In [89]:
for x, z in l_edges:
    try:
        y1.append(data[1][x][z])
    except:
        pass

In [90]:
import numpy as np
print(np.__version__)



1.11.1


In [91]:
y = np.array(y1).reshape(len(l_edges),1)
print(y)

[[3]
 [0]
 [6]
 [0]
 [3]
 [2]
 [0]
 [0]
 [3]
 [7]]


***DERIVE INCIDENCE MATRIX***

In [92]:
def incidence_matrix(A, nodelist=None, edgelist=None, 
                     oriented=True, weight=None):
    if nodelist is None:
        nodelist = l_nodes
    if edgelist is None:
        edgelist = l_edges
    A = np.zeros((len(nodelist),len(edgelist)))
    node_index = dict( (node,i) for i,node in enumerate(nodelist) )
    for ei,e in enumerate(edgelist):
        (u,v) = e[:2]
        if u == v: continue  # self loops give zero column
        try:
            ui = node_index[u]
            vi = node_index[v]
        except KeyError:
            raise NetworkXError('node %s or %s in edgelist '
                                'but not in nodelist"%(u,v)')
        if weight is None:
            wt = 1
        else:
            if DG.is_multigraph():
                ekey = e[2]
                wt = DG[u][v][ekey].get(weight,1)
            else:
                wt = DG[u][v].get(weight,1)
        if oriented:
            A[ui,ei] = -wt
            A[vi,ei] = wt
        else:
            A[ui,ei] = wt
            A[vi,ei] = wt
    return np.asmatrix(A)

In [93]:
print(incidence_matrix("A"))

[[-1. -1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0. -1.  1.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0. -1. -1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  1.  0.  1.  0.]]


***PERFORM CONSISTENCY CHECK***

In [94]:
#derive vector x
int_change = []
for value in change:
    int_change.append(float(value))
    
x = np.array(int_change).reshape(len(l_nodes), 1)
print(x)

[[ 3.]
 [ 1.]
 [-1.]
 [ 1.]
 [ 3.]
 [ 7.]
 [ 3.]
 [ 3.]]


In [95]:
#check if Ay = x relationship holds
Ay = incidence_matrix("A")*y
print (Ay)
if x.all == Ay.all:
    print("consistent y")
else:
    print("inconsistent y")

[[ 3.]
 [ 1.]
 [-1.]
 [ 1.]
 [-3.]
 [-7.]
 [ 3.]
 [ 3.]]
inconsistent y
